In [ ]:
!pip install LLMChainRouter

In [6]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import chain
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
google_api_key = os.environ.get('GOOGLE_API_KEY')

# 确保 API 密钥已设置
if not google_api_key:
    print("没有找到 Google API 密钥，请确保在 .env 文件中设置 GOOGLE_API_KEY")
    exit()

# 1. 初始化 LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=google_api_key,
    temperature=0.3
)

文学作品判断器：

In [ ]:
is_lyric_prompt_template = """
你是一个文本分类器。你的任务是判断给定的文本是否为歌词或文学作品（例如诗歌、小说片段、散文）。
仅回答 "是" 或 "否"。
文本：{text}
答案：
"""
is_lyric_prompt = PromptTemplate(
    template=is_lyric_prompt_template,
    input_variables=["text"]
)

is_lyric_chain = (
    {"text": RunnablePassthrough()} | is_lyric_prompt | llm | {"is_lyric": itemgetter("text")}
)

尝试搜索，但是 langchian 和 vertexai 用不了 grounding，所以这里用 gemini api。

In [2]:
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 5.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.71.1
    Uninstalling google-cloud-aiplatform-1.71.1:
      Successfully uninstalled google-cloud-aiplatform-1.71.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vertexai 1.71.1 requires google-cloud-aiplatform[all]==1.71.1, but you have google-cloud-aiplatform 1.85.0 which is incompatible.


In [12]:
import base64
import os
from google import genai
from google.genai import types

def search_lyric_meaning(context: str, text: str) -> str:
    """
    分析歌词背后的隐喻或典故。

    Args:
        context: 歌曲的背景信息，例如歌曲名称、歌手、专辑等。
        text: 需要分析的歌词文本。

    Returns:
        包含分析结果的字符串。如果无法找到相关信息，则返回提示消息。
    """

    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.0-flash"
    prompt = f"""
    请分析以下歌词的隐喻，引用，背后故事，并结合歌曲的背景信息。尽可能简洁，当你引用信息的时候请标明来源。请你直接返回分析结果。
    背景信息：（{context}）。
    歌词：{text}
    """

    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    tools = [
        types.Tool(google_search=types.GoogleSearch())
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        top_k=40,
        max_output_tokens=8192,
        tools=tools,
        response_mime_type="text/plain",
    )
    
    try:
        response = client.models.generate_content(
            model=model,
            contents=contents,
            config=generate_content_config,
        )
        return response.text
    except Exception as e:
        return f"分析出错：{str(e)}"

# 示例用法
if __name__ == '__main__':
    context = "歌曲：June Gloom，歌手：Camila Cabello，专辑：C, XOXO"
    lyric = "Does she get this wet for you, baby? \n Talk to you in poems and songs, huh, baby?"
    analysis = search_lyric_meaning(context, lyric)
    print(analysis)


好的，没问题。下面是对Camila Cabello的歌曲 "June Gloom" 中歌词的分析：

这首歌词出自Camila Cabello的专辑C, XOXO中的歌曲 "June Gloom"。根据我对歌词的分析，可以总结为以下几点：

*   **隐喻与引用：** 歌词中 "Does she get this wet for you, baby?" 和 "Talk to you in poems and songs, huh, baby?" 运用了反问的修辞手法，暗示了前任的新女友是否也能像她一样带给对方激情和浪漫。这是一种隐晦的比较，暗示自己曾经在感情中的投入和独特性。
*   **背后故事：** 据报道，这首歌的灵感来自于Camila与前男友（很可能是Shawn Mendes）的重逢经历 [2, 3]。歌词表达了对前任和他的新恋情的复杂情感，包括嫉妒、不甘和自我怀疑 [1, 4]。歌名 "June Gloom" 本身指的是一种天气现象，Camila用它来比喻与前任重逢时那种迷茫和不清醒的状态 [3, 5]。
*   **歌曲背景信息：** "June Gloom" 是一首关于旧情难忘的歌曲，Camila在歌曲中展现了更加奔放和真实的自己 [5]。歌词中透露出她对这段关系的反思和挣扎，以及对前任的复杂情感 [6]。

总的来说，这段歌词通过隐喻和反问，表达了Camila对前任恋情的复杂情感，以及对自己在感情中地位的质疑。歌曲的背景故事则揭示了这段情感的来源，以及Camila创作这首歌的动机。

